In [81]:
# import needed libs
import cv2
import math
import numpy as np
import mediapipe as mp
import pandas as pd
from mio import ResultIO
from paths import *
from sklearn.metrics.pairwise import euclidean_distances
import itertools
import json
import matplotlib.pyplot as plt

# Generate manually selected feature plots

In [82]:
def get_data(data, which):
    return data["features"][which]  # list

def dist_between(a, b, mode="3d"):
    # 3d = consider all x y z
    distance = euclidean_distances([np.array(a), np.array(b)])[0, 1]
    return distance

def get_comb(ls, comb_len=2):
    # Get all possible combinations using itertools.combinations
    combinations = list(itertools.combinations(ls, comb_len))
    return combinations

In [83]:
class Extractor:
    def __init__(self, file_path):
        # Read the JSON file
        self.calc_mode = "3d"
        self.f_tips = ["4", "8", "12", "16", "20"]
        self.f_roots = ["1", "5", "9", "13", "17"]
        self.palm = "0"
        with open(file_path, 'r') as file:
            data = json.load(file)
            self.data = data
            return
    
    def extract(self): 
        outlist = []
        # p0x, p0y, p0z. 
        outlist += get_data(self.data, self.palm) # 0 three dims


        for tipidx in range(len(self.f_tips)):
            tip = self.f_tips[tipidx]
            root = self.f_roots[tipidx]
            outlist.append(dist_between(
                get_data(self.data, root), get_data(self.data, tip)))
        
        dist_combs = get_comb(self.f_tips)
        for comb in dist_combs:
            outlist.append(dist_between(
                get_data(self.data, comb[0]), get_data(self.data, comb[1])
            ))
        
        return np.array(outlist)

In [84]:
def loop_directory_concatenate_dataframes(directory_path):
    # List all files in the directory and sort them
    files = sorted(os.listdir(directory_path))

    # Initialize an empty DataFrame
    df = pd.DataFrame()

    for file in files:
        # Check if the file is a JSON file
        if file.endswith('.json'):
            file_path = os.path.join(directory_path, file)

            ext = Extractor(file_path)

            # Call the function to convert the JSON file to a NumPy array
            np_array = ext.extract()

            # Convert the NumPy array to a DataFrame and add a 'type' column
            temp_df = pd.DataFrame(np_array).T  # this T makes sure the array is transformed into a row, instead of a column
            file_name_without_ext = os.path.splitext(file)[0]
            temp_df.insert(0, 'type', file_name_without_ext)

            # Concatenate the temporary DataFrame with the main DataFrame
            df = pd.concat([df, temp_df], ignore_index=True)

    return df

In [85]:
conted = loop_directory_concatenate_dataframes(graph_dir)
conted.to_csv(os.path.join(feats_dir, "feats.csv"), index=False)

# Plot extracted vecs

In [87]:
def read_vid_graph(name, src, start, end): 
    # Read the CSV file into a DataFrame
    df = pd.read_csv(src)

    df = df[df['type'].str.startswith(name)]

    # Get the list of names
    names = df['type'].tolist()

    # Get the data columns and convert them to NumPy arrays
    data = df.loc[:, start:end].to_numpy()

    # Stack the arrays into a matrix
    matrix = np.vstack(data)
    return names, matrix

In [88]:
def plot_spectrogram(specgram, title=None, ylabel="freq_bin"):
    fig, axs = plt.subplots(1, 1)
    axs.set_title(title or "Spectrogram (db)")
    axs.set_ylabel(ylabel)
    axs.set_xlabel("frame")
    im = axs.imshow(specgram, origin="lower", aspect="auto")
    fig.colorbar(im, ax=axs)
    plt.show(block=False)

In [89]:
def save_spectrogram(specgram, title=None, ylabel="freq_bin", dir="./"):
    fig, axs = plt.subplots(1, 1)
    axs.set_title(title or "Spectrogram (db)")
    axs.set_ylabel(ylabel)
    axs.set_xlabel("frame")
    im = axs.imshow(specgram, origin="lower", aspect="auto", cmap='gray_r')
    fig.colorbar(im, ax=axs)
    # plt.show(block=False)
    plt.savefig(os.path.join(dir, '{}.jpg'.format(title)), format='jpeg', dpi=300)
    plt.close()

In [90]:
def save_line(vals, labels, title=None, dir="./"): 
    for labelidx in range(len(labels)): 
        label = labels[labelidx]
        # Plot the arrays
        plt.plot(vals[:, labelidx], label=label)

    # Customize the graph
    plt.xlabel('frame')
    plt.ylabel('val')
    plt.title(title or 'Line Graph Features')
    plt.legend()
    plt.savefig(os.path.join(dir, '{}.jpg'.format(title)), format='jpeg', dpi=300)
    plt.close()

In [91]:
def read_and_save(start, end, suffix): 
    for vd in os.listdir(det_dir): 
        for clip in os.listdir(det_dir + vd + "/"): 
            n, m = read_vid_graph(clip, os.path.join(feats_dir, "feats.csv"), start, end)
            save_spectrogram(m.T, title=clip + suffix, dir=spec_dir)

In [92]:
def read_and_save_line(start, end, suffix, labels): 
    for vd in os.listdir(det_dir): 
        for clip in os.listdir(det_dir + vd + "/"): 
            n, m = read_vid_graph(clip, os.path.join(feats_dir, "feats.csv"), start, end)
            save_line(m, labels, title=clip + suffix, dir=spec_dir)

In [93]:
read_and_save_line('0', '1', '_p', ["p_x", "p_y"])

In [94]:
read_and_save_line('3', '7', '_open', ["thumb", "index", "middle", "ring", "pinky"])

In [ ]:
read_and_save_line('8', '11', '_dist', ["thumb", "index", "middle", "ring", "pinky"])